In [1]:
import deepchem as dc
import os
import sys
os.environ['DGLBACKEND'] = 'pytorch'
import numpy as np
import pandas as pd
import dgl
import torch
from deepchem.feat.graph_data import GraphData
from deepchem.models import MPNNModel
from rdkit import Chem
from rdkit.Chem import Descriptors
import math

import copy
sys.path.append('../')
from utils import *
from save_data import *
from run_experiments import run_experiments
import matplotlib.pyplot as plt
from itertools import product
from IPython.display import display, HTML
display(HTML("<style>.jp-OutputArea-output {display:flex}</style>"))

2024-03-18 17:42:32.263119: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-18 17:42:32.265519: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64
2024-03-18 17:42:32.265529: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df = pd.read_csv('../expt_phys_data/full_dataset_with_scaffolds.csv').fillna('')
# df['len_smiles'] = 0
df['num_atoms'] = 0
df['contains_ring'] = 0

for i in range(len(df)):
    # df.at[i,'len_smiles'] = len(df.at[i,'smiles'])
    mol = Chem.AddHs(Chem.MolFromSmiles(df.at[i,'smiles']))
    df.at[i,'num_atoms'] = len(mol.GetAtoms())
    if(any(chr.isdigit() for chr in df.at[i,'smiles'])):
        df.at[i,'contains_ring'] = 1
df.describe()

,expt,tip3p,cha,zap9,mbondi,asc,igb5,null,paper_split,hilo_split,num_atoms,contains_ring
count,642.000000,642.000000,642.000000,642.000000,642.000000,642.000000,642.000000,642.0,642.000000,642.000000,642.000000,642.000000
mean,-3.803006,-3.486223,-3.966495,-3.760505,-4.453775,-3.954268,-5.192626,0.0,0.373832,0.373832,18.088785,0.501558
std,3.847820,4.186301,4.106919,4.122175,4.589629,4.956833,4.750501,0.0,0.695742,0.695742,7.129944,0.500387
min,-25.470000,-21.762000,-20.490600,-22.504700,-31.910700,-25.772000,-25.222000,0.0,0.000000,0.000000,3.000000,0.000000
25%,-5.727500,-5.421500,-5.628592,-5.857230,-6.782935,-6.201250,-7.420750,0.0,0.000000,0.000000,13.000000,0.000000
50%,-3.530000,-3.053500,-3.358170,-3.010565,-3.823930,-3.172000,-4.427000,0.0,0.000000,0.000000,17.000000,1.000000
75%,-1.215000,-0.480750,-1.298365,-0.749270,-1.120665,-0.390500,-1.732000,0.0,0.000000,0.000000,22.000000,1.000000
max,3.430000,3.335000,4.873780,3.878180,2.518550,4.954000,2.170000,0.0,2.000000,2.000000,44.000000,1.000000


In [3]:
scaffolds = df.groupby('scaffold').size().reset_index(name='count').sort_values('count', ascending=False).reset_index(drop=True)
scaffolds
N = 0
for scaffold in scaffolds.scaffold:
    temp = df[(df.scaffold == scaffold)]
    expt = temp.expt
    num_atoms = temp.num_atoms
    if(expt.mean() < 7.5):
        N += len(expt)
        print(f'{scaffold}, n = {len(expt)}, mean = {expt.mean():.2f}, std = {expt.std():.2f}, range = ({expt.min():.2f},{expt.max():.2f}), num_atoms mean = {num_atoms.mean():.2f}, contains ring = {temp.contains_ring.mean():.2f}')
print(N)

, n = 320, mean = -2.38, std = 3.21, range = (-23.62,3.16), num_atoms mean = 16.21, contains ring = 0.00
c1ccccc1, n = 152, mean = -4.74, std = 3.09, range = (-11.95,1.07), num_atoms mean = 18.99, contains ring = 1.00
c1ccncc1, n = 22, mean = -5.36, std = 1.19, range = (-8.26,-4.01), num_atoms mean = 15.41, contains ring = 1.00
c1ccc(-c2ccccc2)cc1, n = 16, mean = -3.86, std = 2.14, range = (-9.40,-1.96), num_atoms mean = 22.81, contains ring = 1.00
c1ccc2ccccc2c1, n = 13, mean = -5.24, std = 3.11, range = (-10.21,-2.40), num_atoms mean = 22.62, contains ring = 1.00
c1ccc2c(c1)Oc1ccccc1O2, n = 12, mean = -3.71, std = 0.41, range = (-4.53,-3.10), num_atoms mean = 22.00, contains ring = 1.00
C1CCCCC1, n = 9, mean = -1.54, std = 3.11, range = (-5.46,2.11), num_atoms mean = 22.67, contains ring = 1.00
O=c1cc[nH]c(=O)[nH]1, n = 8, mean = -15.46, std = 3.32, range = (-18.72,-9.73), num_atoms mean = 16.12, contains ring = 1.00
C1CCCC1, n = 5, mean = 0.40, std = 3.33, range = (-5.49,2.55), num_

In [96]:
temp = df[(df.scaffold == '')].reset_index(drop=True)
# plt.hist(temp.expt)
# plt.hist(temp.len_smiles)
temp.sort_values(by='expt',ascending=True)#[(temp.num_atoms >21)].describe()

,id,smiles,iupac name,scaffold,expt,tip3p,cha,zap9,mbondi,asc,igb5,null,train,paper_split,hilo_split,num_atoms,contains_ring
133,mobley_4587267,C([C@H]([C@H]([C@@H]([C@@H](CO)O)O)O)O)O,"(2R,3R,4R,5R)-Hexan-1,2,3,4,5,6-hexol",,-23.62,-18.162,-20.49060,-22.50470,-31.910700,-22.823,-22.502,0,True,0,2,26,0
96,mobley_3259411,C(C(CO)O)O,glycerol,,-13.43,-10.140,-12.30710,-13.04060,-17.848200,-13.638,-14.259,0,True,1,2,14,0
193,mobley_6334915,COP(=O)([C@H](C(Cl)(Cl)Cl)O)OC,"(1R)-2,2,2-trichloro-1-dimethoxyphosphoryl-eth...",,-12.74,-13.424,-7.95879,-11.99200,-13.260200,-13.709,-15.464,0,True,0,2,20,0
23,mobley_1708457,CS(=O)(=O)C,methylsulfonylmethane,,-10.08,-10.559,-11.45930,-12.68350,-12.677900,-14.392,-15.336,0,False,2,2,11,0
45,mobley_1963873,CC(=O)NC,N-methylacetamide,,-10.00,-8.276,-8.86721,-7.02540,-8.583730,-9.663,-10.138,0,True,0,2,12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,mobley_4043987,CCCCC(C)C,2-methylhexane,,2.93,2.894,2.35508,2.34462,1.854630,3.133,1.539,0,True,0,0,23,0
206,mobley_6896128,CCCC[C@@H](C)CC,3-methylheptane,,2.97,3.030,2.51700,2.51909,2.031410,3.376,1.690,0,True,0,0,26,0
203,mobley_676247,C(F)(F)(F)F,tetrafluoromethane,,3.12,2.489,2.81522,2.64568,-0.038411,0.592,-3.364,0,True,0,0,5,0
127,mobley_4375719,CCCCCCCCC,nonane,,3.13,3.221,2.74115,2.81726,2.317270,3.817,1.900,0,True,0,0,29,0


In [119]:
# scaffolds = scaffoldsplitter.generate_scaffolds(dataset)
indexes = [7,56]
e = 0
for i in indexes:
    s = []
    for j in range(i):
        s += df.index[(df.scaffold == scaffolds.scaffold[j+e])].tolist()
        
    e+=i
    expt = df.iloc[s].expt
    print(f'n = {len(expt)}, mean = {expt.mean():.2f}, std = {expt.std():.2f}, range = ({expt.min():.2f},{expt.max():.2f})')
scaffolds

n = 544, mean = -3.29, std = 3.26, range = (-23.62,3.16)
n = 98, mean = -6.68, std = 5.33, range = (-25.47,3.43)


,scaffold,count
0,,320
1,c1ccccc1,152
2,c1ccncc1,22
3,c1ccc(-c2ccccc2)cc1,16
4,c1ccc2ccccc2c1,13
...,...,...
58,C1CCCCCC1,1
59,c1cc2c3c(cccc3c1)CC2,1
60,C1CNC1,1
61,C1CCC1,1


In [17]:
scaffolds = scaffoldsplitter.generate_scaffolds(dataset)
print(len(scaffolds))
for i in range(len(scaffolds)):
    expt = df.iloc[scaffolds[i]].expt
    print(f'n = {len(expt)}, mean = {expt.mean():.2f}, std = {expt.std():.2f}, range = ({expt.min():.2f},{expt.max():.2f})')

63
n = 317, mean = -2.39, std = 3.21, range = (-23.62,3.16)
n = 152, mean = -4.74, std = 3.09, range = (-11.95,1.07)
n = 22, mean = -5.36, std = 1.19, range = (-8.26,-4.01)
n = 16, mean = -3.86, std = 2.14, range = (-9.40,-1.96)
n = 13, mean = -5.24, std = 3.11, range = (-10.21,-2.40)
n = 12, mean = -3.71, std = 0.41, range = (-4.53,-3.10)
n = 9, mean = -1.54, std = 3.11, range = (-5.46,2.11)
n = 8, mean = -15.46, std = 3.32, range = (-18.72,-9.73)
n = 5, mean = 0.40, std = 3.33, range = (-5.49,2.55)
n = 5, mean = -11.31, std = 1.96, range = (-14.21,-9.44)
n = 4, mean = -1.93, std = 2.71, range = (-4.44,0.67)
n = 3, mean = -9.44, std = 0.94, range = (-10.27,-8.41)
n = 3, mean = -3.23, std = 0.28, range = (-3.47,-2.92)
n = 3, mean = -8.18, std = 1.83, range = (-10.22,-6.68)
n = 3, mean = -7.58, std = 0.19, range = (-7.77,-7.40)
n = 3, mean = -5.33, std = 0.26, range = (-5.51,-5.04)
n = 3, mean = -2.65, std = 2.96, range = (-4.61,0.75)
n = 3, mean = -16.37, std = 11.74, range = (-25.47,-

In [27]:
scaffolds = scaffoldsplitter.generate_scaffolds(dataset)
indexes = [6,57]
e = 0
for i in indexes:
    s = []
    for j in range(i):
        s += scaffolds[j+e]
    e+=i
    expt = df.iloc[s].expt
    print(f'n = {len(expt)}, mean = {expt.mean():.2f}, std = {expt.std():.2f}, range = ({expt.min():.2f},{expt.max():.2f})')

n = 532, mean = -3.33, std = 3.26, range = (-23.62,3.16)
n = 107, mean = -6.24, std = 5.36, range = (-25.47,3.43)


In [45]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('../expt_phys_data/full_dataset_partitioned.csv')
hilo = pd.read_csv('../expt_phys_data/full_dataset_hilo_with_val.csv')
# Display the first few rows of the dataframe to understand its structure
# df.head()
df['paper_split'] = df.train_val_test
df['hilo_split'] = hilo.train_val_test
df.drop(columns=['train_val_test'],inplace=True)
df

,id,smiles,iupac name,expt,tip3p,cha,zap9,mbondi,asc,igb5,null,train,paper_split,hilo_split
0,mobley_1017962,CCCCCC(=O)OC,methyl hexanoate,-2.49,-3.299,-3.726220,-3.004340,-4.006280,-3.886,-5.059,0,False,2,1
1,mobley_1019269,CCCCO,butan-1-ol,-4.72,-3.232,-3.135990,-3.222270,-5.188390,-3.080,-4.198,0,False,2,0
2,mobley_1034539,c1cc(c(cc1c2cc(c(c(c2Cl)Cl)Cl)Cl)Cl)Cl,"1,2,3,4-tetrachloro-5-(3,4-dichlorophenyl)benzene",-3.04,-1.083,-2.344750,-2.113720,-0.459959,0.899,-1.348,0,False,2,0
3,mobley_1036761,C1CCC(CC1)N,cyclohexanamine,-4.59,-3.953,-1.755250,-0.447715,-0.027103,0.240,-2.215,0,True,1,0
4,mobley_1046331,c1ccc(cc1)OC=O,phenyl formate,-3.82,-5.442,-5.706530,-6.577580,-7.407990,-8.342,-10.084,0,True,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,mobley_9913368,C(=C/Cl)\Cl,"(E)-1,2-dichloroethylene",-0.78,1.024,-0.252349,-0.595880,-0.520564,-0.090,-1.126,0,True,1,0
638,mobley_9942801,CCc1ccc(cc1)C,1-ethyl-4-methyl-benzene,-0.95,-0.575,-1.201310,-0.421314,-0.758853,0.059,-1.641,0,True,0,0
639,mobley_994483,CCBr,bromoethane,-0.74,0.487,-0.821768,-0.062573,-1.089090,-0.095,-0.818,0,True,0,0
640,mobley_9974966,CC(C)SC(C)C,2-isopropylsulfanylpropane,-1.21,0.140,-1.111810,0.499588,-0.765837,0.032,-1.358,0,True,0,0


In [50]:
from rdkit import Chem
from rdkit.Chem.Scaffolds import MurckoScaffold

# Function to generate the scaffold for a given SMILES string
def generate_scaffold(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    scaffold = MurckoScaffold.GetScaffoldForMol(molecule)
    scaffold_smiles = Chem.MolToSmiles(scaffold)
    return scaffold_smiles

# Apply the function to each SMILES string in the dataframe
# df.insert(3,'scaffold', data['smiles'].apply(generate_scaffold))

# Group by the scaffold
scaffold_groups = df.groupby('scaffold').size().reset_index(name='count').sort_values('count', ascending=False)

# # Display the scaffolds with their counts
scaffold_groups
df

,id,smiles,iupac name,scaffold,expt,tip3p,cha,zap9,mbondi,asc,igb5,null,train,paper_split,hilo_split
0,mobley_1017962,CCCCCC(=O)OC,methyl hexanoate,,-2.49,-3.299,-3.726220,-3.004340,-4.006280,-3.886,-5.059,0,False,2,1
1,mobley_1019269,CCCCO,butan-1-ol,,-4.72,-3.232,-3.135990,-3.222270,-5.188390,-3.080,-4.198,0,False,2,0
2,mobley_1034539,c1cc(c(cc1c2cc(c(c(c2Cl)Cl)Cl)Cl)Cl)Cl,"1,2,3,4-tetrachloro-5-(3,4-dichlorophenyl)benzene",c1ccc(-c2ccccc2)cc1,-3.04,-1.083,-2.344750,-2.113720,-0.459959,0.899,-1.348,0,False,2,0
3,mobley_1036761,C1CCC(CC1)N,cyclohexanamine,C1CCCCC1,-4.59,-3.953,-1.755250,-0.447715,-0.027103,0.240,-2.215,0,True,1,0
4,mobley_1046331,c1ccc(cc1)OC=O,phenyl formate,c1ccccc1,-3.82,-5.442,-5.706530,-6.577580,-7.407990,-8.342,-10.084,0,True,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,mobley_9913368,C(=C/Cl)\Cl,"(E)-1,2-dichloroethylene",,-0.78,1.024,-0.252349,-0.595880,-0.520564,-0.090,-1.126,0,True,1,0
638,mobley_9942801,CCc1ccc(cc1)C,1-ethyl-4-methyl-benzene,c1ccccc1,-0.95,-0.575,-1.201310,-0.421314,-0.758853,0.059,-1.641,0,True,0,0
639,mobley_994483,CCBr,bromoethane,,-0.74,0.487,-0.821768,-0.062573,-1.089090,-0.095,-0.818,0,True,0,0
640,mobley_9974966,CC(C)SC(C)C,2-isopropylsulfanylpropane,,-1.21,0.140,-1.111810,0.499588,-0.765837,0.032,-1.358,0,True,0,0


In [26]:
data.to_csv('~/Downloads/scaffolds.csv',index=False)

In [52]:
df.to_csv('../expt_phys_data/full_dataset_with_scaffolds.csv',index=False)

In [ ]:
%%capture output
import time
a = 0
while a < 90:
    a+=1
    print(a)
    time.sleep(1)